- 과적합은 훈련 데이터에 대한 정확도는 높더라도 새로운 데이터(검증,테스트)에 대해서는 제대로 동작하지 않는다.

## 9-1. 데이터의 양 늘리기

- 데이터의 양을 늘릴수록 모델은 데이터의 일반적인 패턴을 학습하여 과적합을 방지할 수 있습니다.
- 데이터의 양이 적을 경우 데이터 증식 또는 증강을 하여 양을 늘린다.

## 9-2. 모델의 복잡도 줄이기


- 딥러닝에서 복잡도는 은닉층의 수나 매개변수의 수 등으로 결정
- 과적합 현상이 포착되었을 때, 인공 신경망 모델에 대해서 할 수 있는 한 가지 조치는 인공 신경망의 복잡도 줄이는 것이다.

### 예시


In [3]:
from torch import nn

In [4]:
class Example_deep(nn.Module) :
    def __init__(self,input_size,hidden_size, num_classes) :
        super(Example_deep,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size,num_classes)
        
    def forward(self,x) :
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        
        return out

- 위의 인공신경망은 3개의 선형 레이어를 가지고 있다.
- 과적합이 발생했다면 아래의 예시처럼 바꿔 복잡도를 줄일 수 있다.

In [5]:
class decrease(nn.Module) :
    def __init__(self,input_size,hidden_size, num_classes) :
        super(decrease,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size,num_classes)
        
    def forward(self, x) :
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## 9-3. 가중치 규제 적용하기

- L1규제 : 가중치(w)에 대해 절대값 합계를 비용 함수에 추가
- L2규제 : 모든 가중치(w)들의 제곱합을 비용 함수에 추가

- L1 규제의 경우 어떤 특성들이 모델에 영향을 주고 있는지를 정확하게 판단하고자 할 때 유용하다. 만약 이런 판단이 필요없다면 경험적으로는 L2 규제가 더 잘 동작하므로 L2 규제 더 권장

In [6]:
from torch import optim

In [7]:
model = Example_deep(10,20,2)
optimizer = optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-5)

- weight_decay 메개변수를 설정하므로서 L2규제 적용

## 9-4 드롭아웃

- 신경망 학습 시에만 사용하고, 예측 시에는 사용하지 않는 것이 일반적
- 학습 시에 인공 신경망이 특정 뉴런 또는 특정 조합에 너무 의존적이게 되는 것을 방지
- 매번 랜덤 선택으로 뉴런들을 사용하지 않으므로 서로 다른 신경망들을 앙상블하여 사요아는 것 같은 효과를 내어 과적합 방지

## 10-1 ReLU & ReLU 변형들

- 시그모이드는 입력의 절대값이 클 경우 기울기가 0에 가까워지므로 역전파 과정 시에 전파시킬 기울기가 사라져 제대로 역전파가 되지 않는 기울기 소실 문제가 발생

- 완화 방법으로 ReLU와 변형 ReLU를 사용한다.

## 10-2. 가중치 초기화

- 가중치 초기화만 적절히 해줘도 기울기 소실 문제 완화 가능

### 1) 세이비어 초기화

- 균등 분포와 정규 분포로 초기화 할 때 두가지 경우로 나뉜다.
- 여러 층의 기울기 분산 사이에 균형을 맞춰서 특정 층이 너무 주목을 받거나 다른 층이 뒤쳐지는 것을 막는다.
- 시그모이드나 하이퍼볼릭 탄젠트와 같은 S자 형태인 활성화 함수와 함께 사용할 때 좋은 성능을 보이지만, ReLU와는 좋은 성능을 보이지 않는다.

### 2) He 초기화

- He초기화 또한 균등분포 또는 정규분포 두가지 경우로 나뉜다.
- 세이비어와 다르게 다음 층의 뉴런 수를 반영하지 않는다.
- ReLu 계열 함수 사용 시에 효율적3

## 10-3 배치 정규화

- 각 층에 들어가는 입력을 평균과 분산으로 정규화하여 학습을 효율적으로 만든다.

### 1) 내부 공변량 변화

- 학습 과정에서 층 별로 입력 데이터 분포가 달라지는 현상
- 이전 층들의 학습에 의해 이전 층의 가중치 값이 바뀌게 되면, 현재 층에 전달되는 입력 데이터의 분포가 현재 층이 학습했던 시점의 분포와 차이가 발생

### 2) 배치 정규화

- 한 번에 들어오는 배치 단위로 정규화하는 것을 의미
- 각 층에서 활성화 함수를 통과하기 전에 수행
- 입력에 대해 평균을 0으로 만들고 정규화 진행
- 정규화된 데이터에 대해 스케일 & 쉬프트 수행
- 학습 시 배치 단위의 평균과 분산들을 차례로 받아 이동 평균과 이동 분산을 저장해놓았다가 테스트할 때는 해당 배치의 평균과 분산을 구하지 않고 구해놓았던 평균과 분산으로 정규화

- 가중치 초기화에 훨씬 덜 민감
- 훨씬 큰 학습률을 사용할 수 있어서 학습 속도 개선
- 미니 배치마다 평균과 표준편차를 계산하므로 훈련 데이터에 일종의 잡음을 넣는 부수 효과로 과적합을 방지하는 효과도 냅니다. 하지만 부수적 효과이므로 드롭 아웃과 함께 사용하는 것이 좋다.

- 미니 배치 크기에 의존적이며 RNN에 적용하기 어렵다는 단점도 있다.